# Missouri Presidential Primary Results

In this notebook, I will dig into the county-level election results for Missouri's Presidential Primary contests, which were held on March 10, 2020.

The plan:

1. Access the results data
2. Prepare it for analysis
3. Compute state, party and candidate level stats
4. Access county-level boundary data
5. Merge county geographic data with the results
6. Create a choropleth map for each party showing the leader's vote share per county

In [3]:
import os
import xml.etree.ElementTree as et
import pandas as pd
import geopandas as gpd
import altair as alt
import requests

## Accessing the data

For every state-wide election, the Missouri Secretary of State publishes county-level results. Initial results are posted after polls close at 7 pm on election night with updated results posted every few minutes until all precincts have reported.

I can see these results via an XML feed available at this url:

In [4]:
results_url = "https://enrarchives.sos.mo.gov/apfeed/apfeed.asmx/GetElectionResults"

In order to access the feed, you need a key provided by SoS Elections Divisions. To keep the key hidden from public view, I have it stored in an environment variable, which is set via [direnv](https://direnv.net/).

In [5]:
payload = {'AccessKey': os.environ.get('SOS_ACCESS_KEY')}

I can fetch this data using the popular [Requests](https://requests.readthedocs.io/en/master/) library.

One trick I've learned is that, for whatever reason, this endpoint will respond with a 520 status code unless I change the default value in the `'User-Agent'` header from `'python-requests/2.23.0'` to match what a standard web browser would send.

In [6]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:73.0) Gecko/20100101 Firefox/73.0'
}

Now I can make a [`GET`](https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods/GET) request for this resource.

In [7]:
r = requests.get(
    results_url, params=payload, headers=headers
)

This returns a [`Response`](https://requests.readthedocs.io/en/master/api/#requests.Response) object. Here's a quick check to make sure the response was ok.

In [8]:
r.ok

True

## Preparing the data

AFAIK: I can't throw XML immediately into a [`pandas.DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html). I need to pre-process the data. For this task, I will use Python's built-in [XML](https://docs.python.org/3.8/library/xml.etree.elementtree.html) module.

At the top of this notebook, I imported the [`ElementTree`](https://docs.python.org/3.8/library/xml.etree.elementtree.html#module-xml.etree.ElementTree) under the alias `et`. In that module is a function named [`fromstring`](https://docs.python.org/3.8/library/xml.etree.elementtree.html#xml.etree.ElementTree.fromstring). I call that function here, passing the `.content` attribute of the response object, `r`.

In [9]:
root = et.fromstring(r.content)

The `fromstring` function returns the top-level or "root" [`Element`](https://docs.python.org/3.8/library/xml.etree.elementtree.html#xml.etree.ElementTree.Element) of the XML, which in this case is `<ElectionResults LastUpdated="3/10/2020 10:52:10 PM"></ElectionResults>`. This is the outermost wrapper of the XML tree.

The name of the tag indicates the kind of data the element contains. It's sort of like a column header in a tabular data format. We can access this label like this:

In [13]:
print(root.tag)

ElectionResults


We can also access the elements "attributes", which is a dictionary of the all the stuff that's inside the first part of the tag. In this case, there's just one attribute: `'LastUpdated'`, which tells us the date and time the results were last updated.

In [14]:
print(root.attrib['LastUpdated'])

3/10/2020 10:52:10 PM


The `Element` class has two other methods I'll make use of:

- [`.find`](https://docs.python.org/3.8/library/xml.etree.elementtree.html#xml.etree.ElementTree.Element.find), which returns the *first* subelement that matches the string you pass in.
- [`.findall`](https://docs.python.org/3.8/library/xml.etree.elementtree.html#xml.etree.ElementTree.Element.findall), which returns *all* of the matching subelements in a list.

In order load the data into a `pandas.DataFrame`, I need to re-format the data into something more like a tabular format. Specifically, I want a list of lists, which is what you get when you read a csv file into Python. Here I will declare a variable that starts out as just an empty list, which I will then fill up with rows of data.

In [15]:
rows = []

As I perform this data transformation, I will also "flatten" the hierarchical structure. For instance, in the original XML data, there is a single element that contains the name of a given county. In the tabular format, the county anme  will be repeated for each row. Same for the party name. In this flattened dataset, each row will represent the number of votes won by a given candidate in a given party primary in a given county.

In order to do this, I need to loop over the elements at each level of the XML's hierarchy, starting with the counties. Here's how to get all the counties:

In [16]:
counties = root.find('ElectionInfo').find('TypeRace').find('Race').findall('Counties')

Since I'm iterating over all the XML elements anyway, I will also sort the candidates within each county's party primary and set which candidate currently has the most votes (aka, the "leader").

This is...not the most elegant way to transform this data. But it works and is pretty straight-forward.

In [18]:
for county in counties:
    for party in county.find('CountyResults').findall('Party'):
        sorted_candidates = sorted(
            [c for c in party.findall('Candidate')],
            key=lambda c: int(c.find('YesVotes').text),
            reverse=True
        )
        county_party_leader = sorted_candidates[0].find('LastName').text
        county_party_total = sum([int(c.find('YesVotes').text) for c in sorted_candidates])
        for candidate in sorted_candidates:
            row = [
                county.find('CountyName').text,
                int(county.find('CountyResults').find('ReportingPrecincts').text),
                int(county.find('CountyResults').find('TotalPrecincts').text),
                party.find('PartyName').text,
                candidate.find('CandidateID').text,
                candidate.find('LastName').text,
                int(candidate.find('YesVotes').text),
                county_party_total,
                candidate.find('LastName').text == county_party_leader,
            ]
            rows.append(row)

Now I can load this data into a `pandas.DataFrame`. In so doing, I will speficy the column headers. For the sake of consistency, I will reuse the element tags from the XML:

In [20]:
results = pd.DataFrame(
    rows,
    columns=[
        'CountyName', 'ReportingPrecincts', 'TotalPrecincts', 'PartyName', 'CandidateID',
        'LastName', 'YesVotes', 'Total', 'IsLeader'
    ]
)

I'll take a look at the columns, their positions, non-null counts and data-types:

In [21]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8816 entries, 0 to 8815
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   CountyName          8816 non-null   object
 1   ReportingPrecincts  8816 non-null   int64 
 2   TotalPrecincts      8816 non-null   int64 
 3   PartyName           8816 non-null   object
 4   CandidateID         8816 non-null   object
 5   LastName            8816 non-null   object
 6   YesVotes            8816 non-null   int64 
 7   Total               8816 non-null   int64 
 8   IsLeader            8816 non-null   bool  
dtypes: bool(1), int64(4), object(4)
memory usage: 559.7+ KB


And take a peek of the actual data:

In [22]:
results.head()

,CountyName,ReportingPrecincts,TotalPrecincts,PartyName,CandidateID,LastName,YesVotes,Total,IsLeader
0,Adair,12,12,Republican,750117286,Donald J Trump,1129,1183,True
1,Adair,12,12,Republican,750117328,Bill Weld,19,1183,False
2,Adair,12,12,Republican,750117330,Joe Walsh,14,1183,False
3,Adair,12,12,Republican,750117353,Uncommitted,14,1183,False
4,Adair,12,12,Republican,750117300,Bob Ely,5,1183,False


## How many people voted in the Missouri's recent presidential primary?

In [71]:
total_votes = results.YesVotes.sum()

In [72]:
print(f'There where {total_votes:,} cast in the Missouri Presidential Primary')

There where 972,777 cast in the Missouri Presidential Primary


## How many people voted in each party?

In [73]:
party_totals = results.groupby('PartyName').YesVotes.sum().reset_index().sort_values(
    'YesVotes', ascending=False
)

In [74]:
party_totals

,PartyName,YesVotes
1,Democratic,660083
4,Republican,309211
3,Libertarian,2256
0,Constitution,716
2,Green,511


## What were the candidate rankings in each party's primary?

In [75]:
candidate_totals = results.groupby(
    ['PartyName', 'CandidateID', 'LastName']
).YesVotes.sum().reset_index().sort_values(
    ['PartyName', 'YesVotes', 'LastName'], ascending=[True, False, True]
)

In [76]:
candidate_totals

,PartyName,CandidateID,LastName,YesVotes
1,Constitution,750117335,Don Blankenship,278
2,Constitution,750117357,Uncommitted,270
0,Constitution,750117333,Don J. Grundmann,168
8,Democratic,750117279,Joseph R. Biden,396826
11,Democratic,750117283,Bernie Sanders,228244
21,Democratic,750117324,Michael R. Bloomberg,9754
10,Democratic,750117282,Elizabeth Warren,8066
3,Democratic,750117274,Tulsi Gabbard,4838
4,Democratic,750117275,Pete Buttigieg,3272
5,Democratic,750117276,Amy Klobuchar,2652
